In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
import random  # for random error injection
# QuantumRegister(n): Creates a register of n quantum bits (qubits)
# ClassicalRegister(n): Creates a register of n classical bits (used to store measurement results)
# Quantum data must be measured to extract classical information
#AerSimulator emulates a real Quantum Computer.
#It can simulate superposition, entanglement, etc.

# --- Define Quantum & Classical Registers ---
q = QuantumRegister(5)             # 3 data qubits + 2 ancillas
c_syndrome = ClassicalRegister(2)  # for syndrome measurement
c_final = ClassicalRegister(1)     # final measurement after correction
backend = AerSimulator()

# --- Step 1: Create encoding, error, and syndrome detection circuit ---
#QuantumCircuit comes into use here. It's one of qiskit's core concepts.
#It contains qubit & bit registers. We can apply logic gates to our qubits.
#Qubit & bit registers are our quantum memory & measurement result info respectively.
#The circuit can be visualised, simulated or run on real Quantum hardware.
def create_syndrome_circuit(error_qubit=None):
    qc = QuantumCircuit(q, c_syndrome)

    # Prepare |+> state = (|0> + |1>) / sqrt(2)
    #The Hadamard gate, h(), creates these superpositions.
    qc.h(q[0])

    # Encode logical qubit using 3-qubit repetition code
    # The CNOT gate, cx(), conditionally flips the target qubit if the control qubit is |1⟩
    # Here, it spreads the logical qubit's state across multiple physical qubits
    #barrier() provides visual/logical separation between steps.
    #It's especially useful in visualisation.
    qc.cx(q[0], q[1])
    qc.cx(q[0], q[2])
    qc.barrier()

    # Inject bit-flip error if specified
    if error_qubit is not None:
        qc.x(q[error_qubit])
    qc.barrier()

    # Syndrome measurement using ancillas q[3] and q[4]
    #measure() measures the quantum bit, collapsing the wavefunction.
    #We get a classical bit of info.
    qc.cx(q[0], q[3])
    qc.cx(q[1], q[3])
    qc.measure(q[3], c_syndrome[0])

    qc.cx(q[1], q[4])
    qc.cx(q[2], q[4])
    qc.measure(q[4], c_syndrome[1])

    return qc

# --- Step 2: Create correction + decoding circuit based on syndrome ---
def create_correction_and_decoding_circuit(syndrome):
    qc = QuantumCircuit(q, c_final)

    # Apply correction based on syndrome bits
    if syndrome == '01':
        qc.x(q[2])  # Error on qubit 2
    elif syndrome == '10':
        qc.x(q[0])  # Error on qubit 0
    elif syndrome == '11':
        qc.x(q[1])  # Error on qubit 1

    qc.barrier()

    # Decode (reverse encoding)
    qc.cx(q[0], q[2])
    qc.cx(q[0], q[1])
    qc.barrier()

    # Measure in X-basis (Hadamard before Z-measurement)
    qc.h(q[0])
    qc.measure(q[0], 0)

    return qc

# --- Step 3: Randomly inject a bit-flip error on one of the 3 data qubits ---
#Going to use the random module here.
error_qubit = random.choice([0, 1, 2])  # Random selection
print(f"\nRandomly injecting bit-flip error on qubit {error_qubit}")

#Now to use transpile from qiskit.
#It'll convert the high-level circuit to something lower-level.
#We can then run it on a specific backend.
#We'll also use run and result from qiskit aer.
syndrome_circuit = create_syndrome_circuit(error_qubit)
tqc = transpile(syndrome_circuit, backend)
result = backend.run(tqc, shots=1024).result()
counts = result.get_counts()
print("Syndrome measurement counts:", counts)

# --- Step 4: Identify most common syndrome (error pattern) ---
# Use Python's max() function to find the most frequent measurement result (the syndrome)
# This tells us which qubit likely experienced a bit-flip error
most_common_syndrome = max(counts, key=counts.get)
print("Most common syndrome:", most_common_syndrome)

# --- Step 5: Run correction and decoding based on syndrome ---
correction_circuit = create_correction_and_decoding_circuit(most_common_syndrome)
tqc_corr = transpile(correction_circuit, backend)
result_corr = backend.run(tqc_corr, shots=1024).result()
counts_corr = result_corr.get_counts()
print("Final measurement after correction and decoding (X-basis):", counts_corr)
# Ideally, we should see approximately equal counts for '0' and '1'
# This indicates successful recovery of the original |+⟩ state (superposition)



Randomly injecting bit-flip error on qubit 1
Syndrome measurement counts: {'11': 1024}
Most common syndrome: 11
Final measurement after correction and decoding (X-basis): {'1': 510, '0': 514}
